In [1]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
import matplotlib.pyplot as plt
import pickle
import heapq
import os

In [2]:
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Activation
from keras.optimizers import RMSprop

In [3]:
path=r'E:\notes\LetsGrowMore\Task04-NextWordPrediction'
os.chdir(path)

In [4]:
path = r'E:\notes\LetsGrowMore\Task04-NextWordPrediction\1661-0.txt'
with open(path, 'r', encoding='utf-8') as file:
    text = file.read().lower()
print('corpus length:', len(text))

corpus length: 581888


In [5]:
import re
# Remove unwanted characters
text = re.sub(r'[^a-zA-Z\s]', '', text)

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [7]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
# Pad the sequences to ensure uniform input length
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [8]:
from tensorflow.keras.layers import Embedding,Dropout
# Split the input and target
X, y = input_sequences[:,:-1], input_sequences[:,-1]
# Convert labels to categorical (one-hot encoded)
y = tf.keras.utils.to_categorical(y, num_classes=total_words)
# Define the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dropout(0.2))
model.add(Dense(total_words, activation='softmax'))
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Train the model
history = model.fit(X, y, epochs=100, verbose=1)
# Save the model
model.save('next_word_prediction_model.h5')

C:\Users\Souymo Tewari\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/100
3057/3057 ━━━━━━━━━━━━━━━━━━━━ 81s 25ms/step - accuracy: 0.0617 - loss: 6.5466
Epoch 2/100
3057/3057 ━━━━━━━━━━━━━━━━━━━━ 75s 24ms/step - accuracy: 0.1127 - loss: 5.6920
Epoch 3/100
3057/3057 ━━━━━━━━━━━━━━━━━━━━ 74s 24ms/step - accuracy: 0.1391 - loss: 5.3361
Epoch 4/100
3057/3057 ━━━━━━━━━━━━━━━━━━━━ 73s 24ms/step - accuracy: 0.1567 - loss: 5.0478
Epoch 5/100
3057/3057 ━━━━━━━━━━━━━━━━━━━━ 73s 24ms/step - accuracy: 0.1690 - loss: 4.7947
Epoch 6/100
3057/3057 ━━━━━━━━━━━━━━━━━━━━ 73s 24ms/step - accuracy: 0.1839 - loss: 4.5408
Epoch 7/100
3057/3057 ━━━━━━━━━━━━━━━━━━━━ 74s 24ms/step - accuracy: 0.2042 - loss: 4.3054
Epoch 8/100
3057/3057 ━━━━━━━━━━━━━━━━━━━━ 74s 24ms/step - accuracy: 0.2214 - loss: 4.0690
Epoch 9/100
3057/3057 ━━━━━━━━━━━━━━━━━━━━ 74s 24ms/step - accuracy: 0.2448 - loss: 3.8595
Epoch 10/100
3057/3057 ━━━━━━━━━━━━━━━━━━━━ 73s 24ms/step - accuracy: 0.2681 - loss: 3.6644
Epoch 11/100
3057/3057 ━━━━━━━━━━━━━━━━━━━━ 73s 24ms/step - accuracy: 0.2925 - loss: 3.47

In [9]:
# Function to predict the next word
def predict_next_word(model, tokenizer, text_seq, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text_seq])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)[0]
    
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
# Example usage:
seed_text = "Once upon a time"
next_word = predict_next_word(model, tokenizer, seed_text, max_sequence_len)
print(f"Predicted next word: {next_word}")

Predicted next word: that


In [20]:
from keras.preprocessing.sequence import pad_sequences
# Function to predict the next word
def predict_next_word(model, tokenizer, text_seq, max_sequence_len):
    # Convert the text sequence to tokens
    token_list = tokenizer.texts_to_sequences([text_seq])[0]
    # Pad the sequence to match the expected input length
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    # Predict the next word
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)[0]
    # Find the word corresponding to the predicted index
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return "Unknown"  # Return a fallback if no word is found
# Example usage:
seed_text = "Once upon a time"
next_word = predict_next_word(model, tokenizer, seed_text, max_sequence_len)
print(f"Predicted next word: {next_word}")

Predicted next word: in
